<a href="https://colab.research.google.com/github/dookda/cmu_py499/blob/main/gee_meji_usc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api
!pip install geemap folium

# Authenticate and initialize the Earth Engine API
import ee

# Provide your Google Cloud project ID
project_id = 'earthengine-380405'

# Authenticate and initialize the Earth Engine API with the project ID
ee.Authenticate()
ee.Initialize(project=project_id)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!rm -r '/content/drive/MyDrive/_DATASET/meji_usc.csv'

In [15]:
import ee

ee.Initialize()

# hex150 = ee.FeatureCollection("projects/earthengine-380405/assets/hex150tb_4326");
hexpolygon = ee.FeatureCollection("projects/ee-sakda/assets/hex_usc");
site = hexpolygon.geometry().bounds()


In [46]:
import ee
import folium
import geemap
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def get_min(img, geom):
    min_val = img.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=geom,
        scale=100,
        maxPixels=1e9
    ).getInfo()
    return min_val['constant']

def get_max(img, geom):
    max_val = img.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=geom,
        scale=100,
        maxPixels=1e9
    ).getInfo()
    return max_val['constant']

def rename_property(feature, params):
    mean_value = feature.get('mean')
    feature = feature.set(params, mean_value)
    feature = feature.set('mean', None)
    return feature

def set_default(feature, params):
    value = ee.Number(feature.get(params))
    return feature.set(params, ee.Algorithms.If(value, value, 0))

def zonal_stat(image, geom, params):
    z_stats = image.reduceRegions(
        collection=geom,
        reducer=ee.Reducer.mean(),
        scale=100,
        tileScale=1,
        crs='EPSG:4326'
    )
    feat = z_stats.map(lambda feature: rename_property(feature, params))
    feat = feat.map(lambda feature: set_default(feature, params))
    return feat

def convert_kelvin_to_celsius(image):
    return image.multiply(0.02).subtract(273.15).rename('LST_Celsius')

def set_day_of_week(image):
    date = ee.Date(image.get('system:time_start'))
    dayOfWeek = date.getRelative('day', 'week')
    dayName = date.format('EEEE')
    return image.set('day_of_week', dayOfWeek).set('date', date.format('YYYY-MM-dd')).set('day_name', dayName)

def prepare_data(image):
    dateImage = set_day_of_week(image)
    projImage = dateImage.reproject(
        crs="EPSG:32647",
        scale=100
    )
    clipImage = projImage.clip(site)
    return clipImage

def show_scatter_plot(feat, params):
    scatter_plot = geemap.ee_chart_by_feature(
        ee.FeatureCollection(feat), x_property=params, y_properties=['usc'], chart_type='ScatterChart',
        options={
            'title': 'Scatter Plot of ' + params + ' vs usc',
            'hAxis': {'title': params},
            'vAxis': {'title': 'usc'},
            'pointSize': 5
        }
    )
    scatter_plot.show()

def loadCollectionPre(start_date, end_date):
    O3 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_O3") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .select('O3_column_number_density') \
        .map(prepare_data) 

    SO2 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_SO2") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .select('SO2_column_number_density') \
        .map(prepare_data) 

    AER_LH = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_AER_LH") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .select('aerosol_optical_depth') \
        .map(prepare_data) 

    CH4 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .select('CH4_column_volume_mixing_ratio_dry_air') \
        .map(prepare_data) 

    HCHO = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_HCHO") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .select('tropospheric_HCHO_column_number_density') \
        .map(prepare_data) 

    NO2 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .select('NO2_column_number_density') \
        .map(prepare_data) 

    CO = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CO") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .select('CO_column_number_density') \
        .map(prepare_data) 
    
    # https://pro.arcgis.com/en/pro-app/latest/arcpy/spatial-analyst/an-overview-of-the-spatial-analyst-functions.htm

    # NDVI = ((NIR - Red) / (NIR + Red))
    # NDMI = (NIR - SWIR1)/(NIR + SWIR1)
    # NDBI = (SWIR - NIR) / (SWIR + NIR)
    # B12=SWIR2, B11=SWIR1, B8=NIR, B4=Red, B3=Green, B2=Blue

    NDVI = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .map(prepare_data) 
    
    NDBI = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .map(prepare_data) 
    
    NDMI = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .map(prepare_data)

    LST = ee.ImageCollection("MODIS/061/MOD11A1") \
        .filterBounds(site) \
        .filterDate(start_date, end_date) \
        .select('LST_Day_1km') \
        .map(prepare_data) \
        .map(convert_kelvin_to_celsius) \
        .select('LST_Celsius') 
    return O3, SO2, AER_LH, CH4, HCHO, NO2, CO, NDVI, NDBI, NDMI, LST

def loadCollectionMedian(start_date, end_date):
    O3, SO2, AER_LH, CH4, HCHO, NO2, CO, NDVI, NDBI, NDMI, LST = loadCollectionPre(start_date, end_date)
    o3 = O3.median()
    so2 = SO2.median()
    aer_lh = AER_LH.median()
    ch4 = CH4.median()
    hcho = HCHO.median()
    no2 = NO2.median()
    co = CO.median()
    ndvi = NDVI.median().normalizedDifference(["B8", "B4"])
    ndbi = NDBI.median().normalizedDifference(["B12", "B8"])
    ndmi = NDMI.median().normalizedDifference(["B8", "B11"])
    lst = LST.median()
    return o3, so2, aer_lh, ch4, hcho, no2, co, ndvi, ndbi, ndmi, lst

def loadCollectionMean(start_date, end_date):
    O3, SO2, AER_LH, CH4, HCHO, NO2, CO, NDVI, NDBI, NDMI, LST = loadCollectionPre(start_date, end_date)
    o3 = O3.mean()
    so2 = SO2.mean()
    aer_lh = AER_LH.mean()
    ch4 = CH4.mean()
    hcho = HCHO.mean()
    no2 = NO2.mean()
    co = CO.mean()
    ndvi = NDVI.mean().normalizedDifference(["B8", "B4"])
    ndbi = NDBI.mean().normalizedDifference(["B12", "B8"])
    ndmi = NDMI.mean().normalizedDifference(["B8", "B11"])
    lst = LST.mean()
    return o3, so2, aer_lh, ch4, hcho, no2, co, ndvi, ndbi, ndmi, lst

def getImageCollectionDayMedian(collection,params):
    monImages = collection.filter(ee.Filter.eq('day_of_week', 0)).mean()
    tueImages = collection.filter(ee.Filter.eq('day_of_week', 1)).mean()
    wedImages = collection.filter(ee.Filter.eq('day_of_week', 2)).mean()
    thuImages = collection.filter(ee.Filter.eq('day_of_week', 3)).mean()
    friImages = collection.filter(ee.Filter.eq('day_of_week', 4)).mean()
    satImages = collection.filter(ee.Filter.eq('day_of_week', 5)).mean()
    sunImages = collection.filter(ee.Filter.eq('day_of_week', 6)).mean()

    mon = zonal_stat(monImages, hexpolygon, params+'mon')
    tue = zonal_stat(tueImages, mon, params+'tue')
    wed = zonal_stat(wedImages, tue, params+'wed')
    thu = zonal_stat(thuImages, wed, params+'thu')
    fri = zonal_stat(friImages, thu, params+'fri')
    sat = zonal_stat(satImages, fri, params+'sat')
    sun = zonal_stat(sunImages, sat, params+'sun')

    return sun

def getImageCollectionDayMean(collection,params):
    print(collection.size().getInfo())
    monImages = collection.filter(ee.Filter.eq('day_of_week', 0)).mean()
    tueImages = collection.filter(ee.Filter.eq('day_of_week', 1)).mean()
    wedImages = collection.filter(ee.Filter.eq('day_of_week', 2)).mean()
    thuImages = collection.filter(ee.Filter.eq('day_of_week', 3)).mean()
    friImages = collection.filter(ee.Filter.eq('day_of_week', 4)).mean()
    satImages = collection.filter(ee.Filter.eq('day_of_week', 5)).mean()
    sunImages = collection.filter(ee.Filter.eq('day_of_week', 6)).mean()

    mon = zonal_stat(monImages, hexpolygon, params+'mon')
    tue = zonal_stat(tueImages, mon, params+'tue')
    wed = zonal_stat(wedImages, tue, params+'wed')
    thu = zonal_stat(thuImages, wed, params+'thu')
    fri = zonal_stat(friImages, thu, params+'fri')
    sat = zonal_stat(satImages, fri, params+'sat')
    sun = zonal_stat(sunImages, sat, params+'sun')

    return sun

# get week from date
def get_week(date= '2021-01-01'):
    date = ee.Date(date)
    return date.get('week')

# get date range from week
def get_date_range(week=1, year=2021):
    start_date = ee.Date.fromYMD(year, 1, 1)
    start_date = start_date.advance(week, 'week')
    end_date = start_date.advance(1, 'week')
    return start_date, end_date

def export_img_to_drive(image, name, folder, scale=100):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder=folder,
        fileNamePrefix=name,
        scale=scale,
        region=site
    )
    task.start()

def export_csv_to_drive(feature_collection, param_name, folder):
    task = ee.batch.Export.table.toDrive(
        collection=feature_collection,
        description=param_name,
        folder=folder,
        fileNamePrefix=param_name,
        fileFormat='CSV'
    )
    task.start()

def corr_chart():
    dir = '/Users/sakdahomhuan/Library/CloudStorage/GoogleDrive-sakda.homhuan@gmail.com/My Drive/'
    listfile = os.path.join(dir, '_GEE')
    for file in os.listdir(listfile):
        df = pd.read_csv(os.path.join(listfile, file))
        # print(df.head())
        columns_of_interest = ['aer_lh', 'bd_h', 'ch4', 'co', 'hcho', 'lst',
                        'no2', 'o3', 'road_w','so2', 'usc', 'ndvi']

        correlation_matrix = df[columns_of_interest].corr()
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
        plt.title('Correlation Matrix')
        # plt.show()

        chart_name = file.replace('.csv', '.png')
        print(chart_name)
        output_dir = os.path.join(listfile, 'chart')
        plt.savefig(os.path.join(output_dir, chart_name))

In [50]:
# ทำ Zonal stat แบบรายวัน
start_date = '2021-11-01'
end_date = '2022-04-30' 

O3, SO2, AER_LH, CH4, HCHO, NO2, CO, NDVI, NDBI, NDMI, LST = loadCollectionPre(start_date, end_date)

featureO3 = getImageCollectionDayMean(O3, 'o3')
featureSO2 = getImageCollectionDayMean(SO2, 'so2')
featureAER_LH = getImageCollectionDayMean(AER_LH, 'aer_lh')
featureCH4 = getImageCollectionDayMean(CH4, 'ch4')
featureHCHO = getImageCollectionDayMean(HCHO, 'hcho')
featureNO2 = getImageCollectionDayMean(NO2, 'no2')
featureCO = getImageCollectionDayMean(CO, 'co')
featureNDVI = getImageCollectionDayMean(NDVI, 'ndvi')
featureNDBI = getImageCollectionDayMean(NDBI, 'ndbi')
featureNDMI = getImageCollectionDayMean(NDMI, 'ndmi')
featureLST = getImageCollectionDayMean(LST, 'lst')

# select columns
featureO3 = featureO3.select(['usc', 'o3mon', 'o3tue', 'o3wed', 'o3thu', 'o3fri', 'o3sat', 'o3sun'])
featureSO2 = featureSO2.select(['so2mon', 'so2tue', 'so2wed', 'so2thu', 'so2fri', 'so2sat', 'so2sun'])
featureAER_LH = featureAER_LH.select(['aer_lhmon', 'aer_lhtue', 'aer_lhwed', 'aer_lhthu', 'aer_lhfri', 'aer_lhsat', 'aer_lhsun'])
featureCH4 = featureCH4.select(['ch4mon', 'ch4tue', 'ch4wed', 'ch4thu', 'ch4fri', 'ch4sat', 'ch4sun'])
featureHCHO = featureHCHO.select(['hchomon', 'hchotue', 'hchowed', 'hchothu', 'hchofri', 'hchosat', 'hchosun'])
featureNO2 = featureNO2.select(['no2mon', 'no2tue', 'no2wed', 'no2thu', 'no2fri', 'no2sat', 'no2sun'])
featureCO = featureCO.select(['comon', 'cotue', 'cowed', 'cothu', 'cofri', 'cosat', 'cosun'])
featureNDVI = featureNDVI.select(['ndvimon', 'ndvitue', 'ndviwed', 'ndvithu', 'ndvifri', 'ndvisat', 'ndvisun'])
featureNDBI = featureNDBI.select(['ndbimon', 'ndbitue', 'ndbiwed', 'ndbithu', 'ndbifri', 'ndbisat', 'ndbisun'])
featureNDMI = featureNDMI.select(['ndmimon', 'ndmitue', 'ndmiwed', 'ndmithu', 'ndmifri', 'ndmisat', 'ndmisun'])
featureLST = featureLST.select(['lstmon', 'lsttue', 'lstwed', 'lstthu', 'lstfri', 'lstsat', 'lstsun'])

# merge featureCollection
feature = featureO3.merge(featureSO2).merge(featureAER_LH).merge(featureCH4).merge(featureHCHO).merge(featureNO2).merge(featureCO).merge(featureNDVI).merge(featureNDBI).merge(featureNDMI).merge(featureLST)

export_csv_to_drive(feature_collection=feature, param_name='usc_o3', folder='_GEE')

print(featureO3.size().getInfo())
print('Finish')
# firstImage = featureO3.first()
# date = ee.Date(firstImage.get('system:time_start'))
# print('Date:', date.getInfo())

# firstImage.getInfo()


2543
5414
Finish


In [42]:
# ทำ Zonal stat แบบรายสัปดาห์
week = range(1, 53)
year = 2021

for w in week:
    try:
        start_date, end_date = get_date_range(w, year)
        print(w, start_date.format('YYYY-MM-dd').getInfo(), end_date.format('YYYY-MM-dd').getInfo())

        O3, SO2, AER_LH, CH4, HCHO, NO2, CO, NDVI, NDBI, NDMI, LST = loadCollectionMedian(start_date, end_date)

        lst = zonal_stat(LST, hexpolygon, 'lst')
        ndvi = zonal_stat(NDVI, lst, 'ndvi')
        o3 = zonal_stat(O3, ndvi, 'o3')
        so2 = zonal_stat(SO2, o3, 'so2')
        aer_lh = zonal_stat(AER_LH, so2, 'aer_lh')
        ch4 = zonal_stat(CH4, aer_lh, 'ch4')
        hcho = zonal_stat(HCHO, ch4, 'hcho')
        co = zonal_stat(CO, hcho, 'co')
        ndbi = zonal_stat(NDBI, co, 'ndbi')
        ndmi = zonal_stat(NDMI, ndbi, 'ndmi')
        no2 = zonal_stat(NO2, ndmi, 'no2')

        export_csv_to_drive(feature_collection=no2, param_name='usc_'+str(w)+'_'+str(year), folder='_GEE/median2022')
    except Exception as e:
        print(f"An error occurred in week {w}: {e}")


1 2021-01-08 2021-01-15
2 2021-01-15 2021-01-22
3 2021-01-22 2021-01-29
4 2021-01-29 2021-02-05
5 2021-02-05 2021-02-12
6 2021-02-12 2021-02-19
7 2021-02-19 2021-02-26
8 2021-02-26 2021-03-05
9 2021-03-05 2021-03-12
10 2021-03-12 2021-03-19
11 2021-03-19 2021-03-26
12 2021-03-26 2021-04-02
13 2021-04-02 2021-04-09
14 2021-04-09 2021-04-16
15 2021-04-16 2021-04-23
16 2021-04-23 2021-04-30
17 2021-04-30 2021-05-07
18 2021-05-07 2021-05-14
19 2021-05-14 2021-05-21
20 2021-05-21 2021-05-28
21 2021-05-28 2021-06-04
22 2021-06-04 2021-06-11
23 2021-06-11 2021-06-18
24 2021-06-18 2021-06-25
25 2021-06-25 2021-07-02
26 2021-07-02 2021-07-09
27 2021-07-09 2021-07-16
28 2021-07-16 2021-07-23
29 2021-07-23 2021-07-30
30 2021-07-30 2021-08-06
31 2021-08-06 2021-08-13
32 2021-08-13 2021-08-20
33 2021-08-20 2021-08-27
34 2021-08-27 2021-09-03
35 2021-09-03 2021-09-10
36 2021-09-10 2021-09-17
37 2021-09-17 2021-09-24
38 2021-09-24 2021-10-01
39 2021-10-01 2021-10-08
40 2021-10-08 2021-10-15
41 2021-1

In [ ]:
# วิเคราะห์ความสัมพันธ์ แบบง่ายๆ สัปดาห์เดียว
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

file_path = '/content/drive/MyDrive/_DATASET/meji_usc.csv'

df = pd.read_csv(file_path)
print(df.columns)

columns_of_interest = ['aer_lh', 'bd_h', 'ch4', 'co', 'hcho', 'lst',
                       'no2', 'o3', 'road_w','so2', 'usc', 'ndvi']

correlation_matrix = df[columns_of_interest].corr()
print(correlation_matrix)
# Plot the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# วิเคราะห์ความสัมพันธ์ของข้อมูล ลูปทุกสัปดาห์
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

dir = '/Users/sakdahomhuan/Library/CloudStorage/GoogleDrive-sakda.homhuan@gmail.com/My Drive/'
listfile = os.path.join(dir, '_GEE')

for file in os.listdir(listfile):
    if not file.endswith('.csv'):
        continue

    df = pd.read_csv(os.path.join(listfile, file))
    columns_of_interest = ['usc', 'co', 'no2', 'o3', 'so2', 'lst', 'ndbi', 'ndmi','ndvi']
    # print(df.head())
    # print(df[columns_of_interest].head())

    f_name = file.split('.')
    correlation_matrix = df[columns_of_interest].corr()
    
    usc_record = correlation_matrix[:1]
    usc_record['file_name'] = f_name[0]
    csv_path = os.path.join(listfile, 'correlation_data.txt')
    if os.path.exists(csv_path):
        usc_record.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        usc_record.to_csv(csv_path, index=False)


    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Matrix ' + str(f_name[0]))
    plt.show()

    output_dir = os.path.join(listfile, 'chart')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    chart_name = f_name[0] + '.png'
    chart_path = os.path.join(output_dir, chart_name)

    if os.path.exists(chart_path):
        os.remove(chart_path)
    plt.savefig(chart_path)
    plt.close()

    print('saved ' + chart_name)


In [47]:
# แสดงผลลัพธ์บนแผนที่

map_center = [18.80, 99]
f = folium.Figure(height=400)
m = folium.Map(location=map_center, zoom_start=12).add_to(f)

google_satellite = folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite',
    overlay=False,
    control=True
).add_to(m)

def showMap(feat, geom, param):
  empty = ee.Image().byte()
  imageUsc = empty.paint( featureCollection=feat, color=param)
  visualization = {
      'min': get_min(imageUsc, geom),
      'max': get_max(imageUsc, geom),
      'palette': ['green', 'yellow', 'red']
  }

  map_id_dict = ee.Image(imageUsc).getMapId(visualization)
  folium.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=param,
      overlay=True,
      control=True,
      opacity=0.5
  ).add_to(m)

showMap(co, site, 'usc')
showMap(no2, site, 'no2')
showMap(ndvi, site, 'ndvi')
showMap(ndmi, site, 'ndmi')
showMap(ndbi, site, 'ndbi')
showMap(lst, site, 'lst')
showMap(o3, site, 'o3')

folium.LayerControl().add_to(m)
m


In [ ]:
# วิเคราะห์ MLR

import os
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# drive.mount('/content/drive')
dir = '/Users/sakdahomhuan/Library/CloudStorage/GoogleDrive-sakda.homhuan@gmail.com/My Drive/'
file_path = os.path.join(dir, '_GEE')

for file in os.listdir(file_path):
    if not file.endswith('.csv'):
        continue

    df = pd.read_csv(os.path.join(file_path, file))
    df = df.dropna(subset=['usc'])

    if df.shape[0] == 0:
        raise ValueError("No samples available after dropping rows with null values.")

    target_variable = 'co'
    columns_of_interest = ['usc', 'ndvi', 'ndbi', 'ndmi', 'lst']

    X = df[columns_of_interest]
    y = df[target_variable]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Forward stepwise regression
    def forward_stepwise_selection(X, y):
        initial_features = []
        remaining_features = list(X.columns)
        best_features = []
        while remaining_features:
            scores_with_candidates = []
            for candidate in remaining_features:
                features = initial_features + [candidate]
                X_train_with_candidate = sm.add_constant(X[features])
                model = sm.OLS(y, X_train_with_candidate).fit()
                score = model.rsquared_adj
                scores_with_candidates.append((score, candidate))
            scores_with_candidates.sort(reverse=True)
            best_new_score, best_candidate = scores_with_candidates[0]
            if initial_features:
                best_old_score = sm.OLS(y, sm.add_constant(X[initial_features])).fit().rsquared_adj
                if best_new_score <= best_old_score:
                    break
            initial_features.append(best_candidate)
            remaining_features.remove(best_candidate)
            best_features.append((best_new_score, best_candidate))
        return best_features

    best_features = forward_stepwise_selection(X_train, y_train)

    # print("Selected features and their adjusted R-squared scores:")
    # for score, feature in best_features:
    #     print(f"{feature}: {score}")

    selected_features = [feature for _, feature in best_features]
    X_train_selected = sm.add_constant(X_train[selected_features])
    X_test_selected = sm.add_constant(X_test[selected_features])
    final_model = sm.OLS(y_train, X_train_selected).fit()

    # print(final_model.summary())

    y_pred = final_model.predict(X_test_selected)
    print(f"R-squared in test set: {final_model.rsquared:.3f} of {file}")
    
